In [681]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.datasets import make_blobs
import random
import json
import time
from tabulate import tabulate

from kdtreen import *
from helper import *

In [682]:
savefile = "mst_data3.json"
to_plot = False
no_points = 1000
no_centres = 3

In [683]:
if no_points > 999:
    to_plot = False

In [684]:
from sklearn.datasets import make_blobs, make_circles

X, Y = make_blobs(n_samples=no_points, centers=no_centres, random_state=42)
# points = list(set([(round(x, 1), round(y, 1)) for x, y in X]))
points = list(set([(x, y) for x, y in X]))

In [685]:
# X, y = make_circles(n_samples=no_points, factor=0.5, noise=0.05)
# points = list(set([(x, y) for x, y in X]))

In [686]:
cordmap = {point: i for i, point in enumerate(points)}

In [687]:
dcran_start_time = time.time()
i_neighbors_dict = {}
tree = KDTree()
G = nx.Graph()
cached_count = 1 - 1

In [688]:
def build(points):
    global cordmap, i_neighbors_dict, tree, G
    limit_dist = min(4**4, math.ceil(math.log2(len(points))))
    tree.root = tree.build(points)

    for coord, index in cordmap.items():
        G.add_node(index, pos=coord)
    for point in points:
        i_neighbors_dict[point] = i_neighbors(tree, point, limit_dist)

    return tree, G

In [689]:
def cached_i_neighbor(tree, dis):
    global i_neighbors_dict
    global points, cached_count
    if dis > cached_count:
        for point in points:
            i_neighbors_dict[point] = i_neighbors(tree, point, dis + 1)

    cached_count = max(cached_count, dis)-1
    return

In [690]:
def add_edges(G, i, pointi, points):
    for dis, pointj in points:
        if pointi == pointj:
            continue
        G.add_edge(i, cordmap[pointj], weight=dis)
    return G

In [691]:
def dcran(points):
    global tree
    global G, cached_count
    tree, G = build(points)
    k = 1
    visited = set()
    while math.factorial(k) <=math.ceil (math.log2(len(points))):

        for pointi, i in cordmap.items():
            points_to_con = []
            dis1 = k - 1
            dis2 = min(math.factorial(k), int(math.log2(len(points)))) - 1
            for dis_hand in (dis1, dis2):
                if dis_hand in visited:
                    continue
                if cached_count < dis_hand:
                    cached_i_neighbor(tree, dis_hand + 1)
                    # print("cached", dis_hand)
                    # print(cached_count, i_neighbors_dict[pointi])
                # try :
                pointj = i_neighbors_dict[pointi][dis_hand]
                # except:
                # print("error")
                # print(dis_hand,cached_count  , pointi, i_neighbors_dict[pointi] , len(i_neighbors_dict[pointi]))
                # pointj = ith_nearest_neighbor(tree, pointi, dis_hand + 1)

                # eudis = euclidean_distance(pointi, pointj)
                # points_to_con.append((eudis , pointj ))
                points_to_con.append(pointj)

            if random.randint(0, 2**64) % len(points) <= k**k**k:
                pointl_cord = points[random.randint(0, len(points) - 1)]
                dis = euclidean_distance(pointi, pointl_cord)
                points_to_con += [(dis, pointl_cord)]
            G = add_edges(G, i, pointi, points_to_con)

        if count_clusters(G) == 1:
            break

        visited.add(dis1)
        visited.add(dis2)
        k += 1
    # for _ in range(int(math.log2(len(cordmap)))Applied Soft Computing Journal):
    #     idi, idj = random.randint(0, len(cordmap) - 1), random.randint(
    #         0, len(cordmap) - 1 )
    #     if idi == idj:
    #         continue
    #     dis = euclidean_distance(points[idi], points[idj])
    #     G.add_edge(cordmap[points[idi]], cordmap[points[idj]], weight=dis)
    print(G)
    print(count_clusters(G), "clusters", k, "k")
    graphify(G, to_plot)


dcran(points)

Graph with 1000 nodes and 3749 edges
1 clusters 3 k


In [692]:
math.log2(len(points))

9.965784284662087

In [693]:
cached_i_neighbor(tree, 2)
x = i_neighbors_dict[points[0]]
print(len(x))
print(x)
print(cached_count)
print(i_neighbors_dict[points[0]])

7
[(0.2967450935547579, (-9.433548326053225, -5.945789682036673)), (0.34690438870803664, (-9.303867451424212, -5.996064196914897)), (0.5569944307356457, (-9.052296737165307, -6.032687916125525)), (0.7273122656328879, (-8.858927001527057, -6.132199337158413)), (0.933443503785189, (-8.773241858105056, -6.666815885903008)), (0.979272007109642, (-9.4099147138976, -7.1661138151560575)), (1.0012403999302508, (-8.754799660797016, -6.76508351412853))]
4
[(0.2967450935547579, (-9.433548326053225, -5.945789682036673)), (0.34690438870803664, (-9.303867451424212, -5.996064196914897)), (0.5569944307356457, (-9.052296737165307, -6.032687916125525)), (0.7273122656328879, (-8.858927001527057, -6.132199337158413)), (0.933443503785189, (-8.773241858105056, -6.666815885903008)), (0.979272007109642, (-9.4099147138976, -7.1661138151560575)), (1.0012403999302508, (-8.754799660797016, -6.76508351412853))]


In [694]:
num_nodes = G.number_of_nodes()
conedge = num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")

Total number of nodes: 1000
Total number of edges: 3749
Total sum of edge weights: 10582.114866329452


In [695]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")
graphify(mst, to_plot)

In [696]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()


foundw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time
print(f"Elapsed time: {dcran_elapsed_time:.2f} seconds")

Total number of nodes: 1000
Total number of edges: 999
Total sum of edge weights: 173.3802682041429
Elapsed time: 0.36 seconds


In [697]:
stmst_start_time = time.time()
G = nx.Graph()

for pointi, i in cordmap.items():
    G.add_node(i, pos=pointi)
    for pointj, j in cordmap.items():
        if i != j:
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, j, weight=dis)

In [698]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")
graphify(mst, to_plot)

In [699]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
realw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
stmst_end_time = time.time()
stmst_elapsed_time = stmst_end_time - stmst_start_time

Total number of nodes: 1000
Total number of edges: 999
Total sum of edge weights: 169.79749951393245


In [700]:
print(stmst_elapsed_time)

4.760290622711182


In [701]:
percentage_error = ((foundw - realw) / realw) * 100
speedup = round((stmst_elapsed_time / dcran_elapsed_time), 2)

# Format the output for readability
formatted_output = f"Real Weight: {realw}  Found Weight: {foundw}  no of Edge: {conedge} Percentage Error: {percentage_error:.2f}%"
print(formatted_output, "speedup", speedup)

Real Weight: 169.79749951393245  Found Weight: 173.3802682041429  no of Edge: 3749 Percentage Error: 2.11% speedup 13.21


In [702]:
! notify-send "DCRAN" "Done"

In [703]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)
currres = []
speedup = round((stmst_elapsed_time / dcran_elapsed_time), 2)
loaded_data.append(
    [
        no_points,
        no_centres,
        foundw,
        realw,
        conedge,
        percentage_error,
        dcran_elapsed_time,
        stmst_elapsed_time,
        speedup,
    ]
)

# Save the updated dictionary back to the JSON file
with open(savefile, "w") as f:
    json.dump(loaded_data, f)

[[100, 3, 69.5834605708583, 61.12651682979382, 217, 13.835147460819272, 0.3657097816467285, 0.15007662773132324, 0.41], [1000, 3, 178.1296547074689, 169.79749951393245, 2073, 4.907113012493306, 0.13645148277282715, 2.6751606464385986, 19.61], [10000, 3, 560.4336576184252, 551.2093922986705, 34115, 1.6734593874185122, 1.4019782543182373, 461.52739453315735, 329.2], [5000, 3, 405.3639353572892, 388.5071736203694, 10298, 4.338854693424907, 0.656238317489624, 84.39165115356445, 128.6], [10000, 3, 563.1719582190365, 551.2093922986705, 26928, 2.1702398557613956, 1.42179536819458, 501.7644808292389, 352.91], [100, 3, 69.4494042709885, 61.12651682979382, 283, 13.615837893020585, 0.04594159126281738, 0.029672622680664062, 0.65], [100, 3, 0, 61.12651682979382, 0, -100.0, 0.07219552993774414, 0.08635163307189941, 1.2], [100, 3, 0, 61.12651682979382, 0, -100.0, 0.05329132080078125, 0.04036855697631836, 0.76], [100, 3, 0, 61.12651682979382, 0, -100.0, 0.05528092384338379, 0.032852888107299805, 0.59

In [704]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)

[[100, 3, 69.5834605708583, 61.12651682979382, 217, 13.835147460819272, 0.3657097816467285, 0.15007662773132324, 0.41], [1000, 3, 178.1296547074689, 169.79749951393245, 2073, 4.907113012493306, 0.13645148277282715, 2.6751606464385986, 19.61], [10000, 3, 560.4336576184252, 551.2093922986705, 34115, 1.6734593874185122, 1.4019782543182373, 461.52739453315735, 329.2], [5000, 3, 405.3639353572892, 388.5071736203694, 10298, 4.338854693424907, 0.656238317489624, 84.39165115356445, 128.6], [10000, 3, 563.1719582190365, 551.2093922986705, 26928, 2.1702398557613956, 1.42179536819458, 501.7644808292389, 352.91], [100, 3, 69.4494042709885, 61.12651682979382, 283, 13.615837893020585, 0.04594159126281738, 0.029672622680664062, 0.65], [100, 3, 0, 61.12651682979382, 0, -100.0, 0.07219552993774414, 0.08635163307189941, 1.2], [100, 3, 0, 61.12651682979382, 0, -100.0, 0.05329132080078125, 0.04036855697631836, 0.76], [100, 3, 0, 61.12651682979382, 0, -100.0, 0.05528092384338379, 0.032852888107299805, 0.59

In [705]:
headers = [
    "Points",
    "Centres",
    "DCRAN Wt",
    "kruskal Wt",
    "Edge count",
    "Acc(%)",
    "DCRAN Time (s)",
    "STMST Time (s)",
    "Speedup",
]

# Format the data as a table using tabulate
table_str = tabulate(
    loaded_data,
    headers,
    tablefmt="pipe",
    floatfmt=(".0f", ".0f", ".1f", ".1f", ".0f", ".2f", ".2f", ".2f", ".2f"),
)
print(table_str)

|   Points |   Centres |   DCRAN Wt |   kruskal Wt |   Edge count |   Acc(%) |   DCRAN Time (s) |   STMST Time (s) |   Speedup |
|---------:|----------:|-----------:|-------------:|-------------:|---------:|-----------------:|-----------------:|----------:|
|      100 |         3 |       69.6 |         61.1 |          217 |    13.84 |             0.37 |             0.15 |      0.41 |
|     1000 |         3 |      178.1 |        169.8 |         2073 |     4.91 |             0.14 |             2.68 |     19.61 |
|    10000 |         3 |      560.4 |        551.2 |        34115 |     1.67 |             1.40 |           461.53 |    329.20 |
|     5000 |         3 |      405.4 |        388.5 |        10298 |     4.34 |             0.66 |            84.39 |    128.60 |
|    10000 |         3 |      563.2 |        551.2 |        26928 |     2.17 |             1.42 |           501.76 |    352.91 |
|      100 |         3 |       69.4 |         61.1 |          283 |    13.62 |             0.05 |